In [2]:
from graph.collabgraph import CollabGraph
from graph.fbgraph import FBGraph
from graph.enrongraph import EnronGraph
from graph.twittergraph import TwitterGraph
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, average_precision_score, auc
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support
from sklearn.ensemble import AdaBoostClassifier

In [3]:
file_names = {'southafrica': '0',
              'fb': 'FacebookFilteredAdj_90Days_6ActiveTimes_30OutInDeg.mat',
              'enron': 'EnronDirectedWithCc_7days.mat',
              'collab': 'citation2Filtered.mat'}
data_root = '/Users/tomfw/Downloads/DataShared/'

In [5]:
# graph = CollabGraph.load_collab_graph(data_root + file_names['collab'])
# graph = FBGraph.load_fb_graph(data_root + file_names['fb'])
#graph = EnronGraph.load_enron_graph(data_root + file_names['enron'])
graph = TwitterGraph.rt_graph_from_json(data_root, 0)

In [5]:
sgs = graph.subgraphs_of_length(days=4)
print(len(sgs))
for sg in sgs:
    print(sg.nx_graph.number_of_edges())

(datetime.datetime(2014, 3, 15, 0, 28, 54), datetime.datetime(2014, 3, 19, 0, 28, 54))


(datetime.datetime(2014, 3, 19, 0, 28, 54), datetime.datetime(2014, 3, 23, 0, 28, 54))


(datetime.datetime(2014, 3, 23, 0, 28, 54), datetime.datetime(2014, 3, 27, 0, 28, 54))


(datetime.datetime(2014, 3, 27, 0, 28, 54), datetime.datetime(2014, 3, 31, 0, 28, 54))


(datetime.datetime(2014, 3, 31, 0, 28, 54), datetime.datetime(2014, 4, 4, 0, 28, 54))


...

(datetime.datetime(2014, 6, 11, 0, 28, 54), datetime.datetime(2014, 6, 15, 0, 28, 54))


23
1544
2076
1114
1563


1812
2645
2839
4740
3028
2571
3042
4057
7109


18764
3751
1061
2271
1578
1009
1130
939


647
461


In [6]:
g_0 = graph.subgraph_within_dates(sgs[0].min_date, sgs[20].max_date)
print(sgs[0].min_date, sgs[6].max_date)

(datetime.datetime(2014, 3, 15, 0, 28, 54), datetime.datetime(2014, 4, 12, 0, 28, 54))


In [8]:
g_1 = graph.subgraph_within_dates(sgs[21].min_date, sgs[21].max_date)
g_2 = graph.subgraph_within_dates(sgs[22].min_date, sgs[22].max_date)
print(sgs[7].min_date, sgs[7].max_date)
print(sgs[8].min_date, sgs[8].max_date)

(datetime.datetime(2014, 4, 12, 0, 28, 54), datetime.datetime(2014, 4, 16, 0, 28, 54))
(datetime.datetime(2014, 4, 16, 0, 28, 54), datetime.datetime(2014, 4, 20, 0, 28, 54))


In [8]:
repeat_edges = 0
repeat_times = 0
for u, v, data in g_1.nx_graph.edges_iter(data=True):
    if g_0.nx_graph.has_edge(u, v):
        repeat_edges += 1
        for time in data['timestamps']:
            for time2 in g_0.nx_graph.edge[u][v]['timestamps']:
                if time == time2:
                    repeat_times += 1
print("%d edges repeated\n%d timestamps repeated" % (repeat_edges, repeat_times))

166 edges repeated
0 timestamps repeated


In [9]:
print g_0.nx_graph.number_of_edges()
print g_1.nx_graph.number_of_edges()
print g_2.nx_graph.number_of_edges()

57397
647
461


In [415]:
# nx.write_edgelist(g_0.nx_graph, 'fb-fucked.edgelist')
# g_0.save_edgelist('twitter_edges.txt')

In [10]:
g_0.embeddings = None

In [12]:
g_0.load_embeddings('twitter-n2v6.txt', 128)

In [13]:
g_1.embeddings = g_0.embeddings
g_1.emb_cols = g_0.emb_cols

g_2.embeddings = g_0.embeddings
g_2.emb_cols = g_0.emb_cols

In [13]:
train_pairs = g_0.make_pairs_with_edges(g_1, .5, enforce_has_embeddings=True, enforce_non_edge=False)
test_pairs = g_1.make_pairs_with_edges(g_2, .5, enforce_non_edge=False, enforce_has_embeddings=True)

Rejected 0 nodes.


Rejected 0 nodes.


In [15]:
df_train, y_train = g_0.to_dataframe(pairs=train_pairs, label_graph=g_1)
rf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2, random_state=0, n_jobs=-1)
# rf = SVC(kernel='linear', probability=True)
# rf = AdaBoostClassifier(n_estimators=500)
fields = g_0.emb_cols
x_train = df_train.loc[:, fields]
classifier = rf.fit(x_train, y_train)

Precomputing katzes....


	1006 pairs checked and 1006 pairs in dataframe


In [16]:
df_test, y_test = g_1.to_dataframe(pairs=test_pairs, label_graph=g_2)

Precomputing katzes....


	696 pairs checked and 696 pairs in dataframe


In [16]:
# fields = g_1.emb_cols
x_test = df_test.loc[:, fields]
pred = classifier.predict_proba(x_test)
print("Prediction made.... Done")

Prediction made.... Done


In [17]:
print(roc_auc_score(y_test, pred[:, 1]))

0.795043929185


In [18]:
pr_curve = precision_recall_curve(y_test, pred[:, 1])

In [19]:
print auc(pr_curve[1], pr_curve[0], reorder=True)
print average_precision_score(y_test, pred[:, 1])

0.824401494469
0.823967320857


In [532]:
bin_labels = []
for p in pred[: , 1]:
    if p > .5:
        bin_labels.append(1)
    else:
        bin_labels.append(0)

In [533]:
pr, re, fs, su = precision_recall_fscore_support(y_test, bin_labels, average='macro')
print pr
print re

0.74877946715
0.735632183908


In [1]:
# sgs = graph.subgraphs_of_length(periods=1)
# tg = TwitterGraph.rt_graph_from_json(data_root, 0)
# sgs = graph.subgraphs_of_length(days=7)

In [2]:
# confirm that no edge/timestamps are repeated in different subgraphs
for i, sg in enumerate(sgs):
    if i > 0:
        n_edges = 0
        n_time_stamps = 0
        prev = sgs[i - 1]
        for u, v, data in prev.nx_graph.edges_iter(data=True):
            if sg.nx_graph.has_edge(u, v):
                n_edges += 1
                for time in data['timestamps']:
                    for time2 in sg.nx_graph.edge[u][v]['timestamps']:
                        if time == time2:
                            n_time_stamps += 1
                       # delta = time2 - time
                       # if delta.days < .5:
                       #     print time, time2, prev.min_date, sg.max_date
        print("%d - %d\n\t%d edges, %d timestamps" % (i, i -1, n_edges, n_time_stamps))
        print(sg.min_date,sg.max_date)
        print(prev.min_date,prev.max_date)

NameError: name 'sgs' is not defined

In [21]:
print sgs[9].nx_graph.number_of_edges()

2571
